In [ ]:
# default_exp core

# CircuitPython_pico_pi_controller

> The `pico_pi_controller` module contains CircuitPython classes to manage multiple Raspberry Pi SBCs from a Raspberry Pi Pico, using i2c.

## About

The classes in this module inherit from Adafruit's `busio` & `adafruit_bus_device` I2C classes for CircuitPython. `busio` is distributed with [CircuitPython](https://circuitpython.org), and `adafruit_bus_device` is distributed with the [CircuitPython Library Bundle](https://circuitpython.org/libraries).

Broadcom SoC hardware I2C is utilized both for the controlling MCU (the Pico) and the controlled SBCs (Raspberry Pi 4/3+/Zero). I2C baudrate is limited from the controller (the Pico) to reduce transmit/receive errors.

Controller programs may be written with hard-coded I2C peripheral addresses of connected SBCs (called 'devices' by these classes). Or, autodiscovery may be used so that devices may be added without changing software on the controller.

For example:

<code>
from pico_pi_controller import *
    
mycontroller = PPController()

mycontroller.addDevice('0x13')
mycontroller.listdevices()
</code>

This example will initialize I2C on the controller using the board defaults (I2C bus 1), retrieve host information & statistics from the SBC at I2C address 0x13, then return a list of device information.

## Module

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
from sys import byteorder, modules
#from rtc import RTC #can't do without MCU CircuitPython
#from time import sleep, datetime, struct_time
from time import sleep
from board import SCL,SDA
from busio import I2C
from adafruit_bus_device.i2c_device import I2CDevice
try:
    from adafruit_itertools.adafruit_itertools import chain
except:
    from itertools import chain
try:
    import adafruit_logging as logging
except:
    import logging

In [ ]:
#export
polling_interval = 2
poll_fail_max = 11
#IDENTITY = bytearray('ppc-daemon','utf-8') # of RPi devices
IDENTITY = bytearray(([ord(c) for c in list('ppc-daemon')]))
"""identifier string used by RPi devices"""

'identifier string used by RPi devices'

In [ ]:
#export
#IDF = bytearray('I','utf-8') # request to send identification
#HOS = bytearray('H','utf-8') # request to send hostname
#LOD = bytearray('L','utf-8') # request to send load
#TIM = bytearray('T','utf-8') # request to send datetime
#PEN = bytearray('P','utf-8') # request to send MCU pin connected to RPi PEN

IDF = bytearray(); IDF.append(ord('I')) # request to send identification
HOS = bytearray(); HOS.append(ord('H')) # request to send hostname
LOD = bytearray(); LOD.append(ord('L')) # request to send load
TIM = bytearray(); TIM.append(ord('T')) # request to send datetime
PEN = bytearray(); PEN.append(ord('P')) # request to send MCU pin connected to RPi PEN

In [ ]:
#export
class PPDevice(I2CDevice):
    """Represents an I2C peripheral device known to a `PPController`
    and stores data from those peripherals identified as PPC devices."""
    def __init__(self, i2c, device_address, **kwargs):
        super().__init__(i2c,device_address,**kwargs)
        #self.device_address = device_address
        self.hostname = "unknown"
        self.bosmang = False
        self.load = ""
        self.pen = 0
        """MCU board pin connected to RPi PEN pin"""
        self.retries = 0
        """retry count before I2CDevice is considered 'other', i.e. not a PPC device."""
        
    def getHostname(self): 
        return self.hostname

In [ ]:
#export
class PPController(I2C):
    """Represents one of the system's I2C busses and tracks which I2C
    peripherals are `PPDevice`s."""
    def __init__(self, scl=SCL, sda=SDA, frequency=4800, bosmang=None, UART_RX=None, UART_TX=None): 
        self.scl = scl
        self.sda = sda
        self.frequency=frequency
        super().__init__(scl, sda, frequency=frequency, timeout=10000)
        self.bosmang = bosmang
        """PPDevice hostname selected to recieve datetime from, send control instructions,
        have UART connected, etc."""
        self.UART_RX = UART_RX
        """board pin RX for passthru from bosmang"""
        self.UART_TX = UART_TX
        """board pin TX for passthru from bosmang"""
        
        self.devices = []
        self.noident = []
        self.othrdev = []
        
        self.datetime = 0
        """future use for receiving datetime from bosmang"""

    def addDevice(self, *args):
        self.devices.append(PPDevice(i2c=self,device_address=args[0]))


    def scanForNew(self):
        while not self.try_lock():
            pass
        for addr in self.scan():
            if not any(d.device_address == addr 
                       for d in chain(self.devices,self.noident,self.othrdev)):
                self.noident.append(PPDevice(i2c=self,device_address=addr))
        self.unlock()
                
    def identifyNew(self):
        for d in self.noident:
            with d:
                msg = bytearray(len(IDENTITY))
                try:
                    d.write_then_readinto(IDF,msg)
                except:
                    pass
            if msg == IDENTITY:
                self.devices.append(self.noident.pop(self.noident.index(d)))
            else:
                d.retries += 1
                if d.retries >= poll_fail_max:
                    self.othrdev.append(self.noident.pop(self.noident.index(d)))
                
    
    def autoAddDevices(self):
        scanForNew()
        identifyNew()
                
    def listdevices(self):
        return self.devices

UART_RX # 
UART_TX # board pin TX for passthru for bosmang

<code>
    def operation(self):
        for device in self._devices:
            device.operation()
</code>

## Appendix

In [ ]:
from nbdev.export import notebook2script
notebook2script()

Converted 00_core.ipynb.
Converted index.ipynb.
